In [36]:
#take organized solution, create a class to represent a plan with initial state, actions and goals
#we have to start by iteratining on the organized solutions we have, and search the corrisponding problem pddl file in the dataset folder to extract initial state and goals
#actions can be taken from the .SOL file

import os
import re

class Plan:
    def __init__(self, problem_id, plan_name, initial_state, actions, goals):
        self.problem_id = problem_id
        self.plan_name = plan_name
        self.initial_state = initial_state
        self.actions = actions
        self.goals = goals

    def __repr__(self):
        return (f"Plan(problem_id={self.problem_id},\n"
                f"     plan_name={self.plan_name},\n"
                f"     initial_state={self.initial_state},\n"
                f"     actions={self.actions},\n"
                f"     goals={self.goals})")

def parse_pddl(pddl_file_path):
    """
    Parses a PDDL file to extract initial state and goals.
    This simple parser iterates through the file line by line.
    """
    initial_state = []
    goals = []
    mode = None
    try:
        with open(pddl_file_path, 'r') as f:
            for line in f:
                line_stripped = line.strip()
                # Start of initial state block
                if line_stripped.startswith("(:init"):
                    mode = "init"
                    # If there is content on the same line
                    remainder = line_stripped[len("(:init"):].strip()
                    if remainder and remainder != "(" and not remainder.startswith(";"):
                        remainder = remainder.replace("(","").replace(")","")
                        initial_state.append(remainder)
                    continue
                # Start of goal block
                elif line_stripped.startswith("(:goal"):
                    mode = "goal"
                    remainder = line_stripped[len("(:goal"):].strip()
                    # Handle goal starting with (and ... if needed
                    if remainder.startswith("(and"):
                        remainder = remainder[len("(and"):].strip()
                        if remainder and remainder != "(" and not remainder.startswith(";"):
                            remainder = remainder.replace("(","").replace(")","")
                            goals.append(remainder)
                    continue
                # End of a block
                elif line_stripped == ")" or line_stripped == ")))":
                    mode = None
                    continue

                # Append lines based on current block
                if mode == "init":
                    if line_stripped and not line_stripped.startswith(";"):
                        line_stripped = line_stripped.replace("(","").replace(")","")
                        if line_stripped:
                            initial_state.append(line_stripped)
                elif mode == "goal":
                    if line_stripped and not line_stripped.startswith(";"):
                        line_stripped = line_stripped.replace("(","").replace(")","")
                        if line_stripped:
                            goals.append(line_stripped)
    except Exception as e:
        print(f"Error parsing PDDL file {pddl_file_path}: {e}")
        raise Exception
    return initial_state, goals

def parse_sol(sol_file_path):
    """
    Parses a SOL file to extract the list of actions.
    Skips commented lines.
    """
    actions = []
    try:
        with open(sol_file_path, 'r') as f:
            for line in f:
                line_stripped = line.strip()
                # Skip comments (lines starting with ;)
                if line_stripped.startswith(";") or not line_stripped:
                    continue
                line_stripped = line_stripped.replace("(","").replace(")","")
                actions.append(line_stripped)
    except Exception as e:
        print(f"Error parsing SOL file {sol_file_path}: {e}")
    return actions

# Base directories for organized solutions and dataset PDDL files.
# Assumes the script is run from /home/rsignoroni/fastdownward_docker/
solutions_organized_dir = "../datasets/gr_logistics/solutions_organized"
dataset_pddl_base = "../datasets/gr_logistics/problems"  # adjust as needed

plans = []
missing_count=0
# Iterate over each problem directory inside solutions_organized
for problem in os.listdir(solutions_organized_dir):
    problem_dir = os.path.join(solutions_organized_dir, problem)
    if not os.path.isdir(problem_dir):
        continue

    sols_dir = os.path.join(problem_dir, "sols")
    if not os.path.exists(sols_dir):
        print(f"No 'sols' directory for problem {problem} in organized solutions.")
        continue

    # For simplicity, pick the first SOL file we find
    sol_files = [f for f in os.listdir(sols_dir) if f.upper().endswith(".SOL")]
    if not sol_files:
        print(f"No SOL files found in {sols_dir}")
        continue
    temp = []
    print(sol_files)
    for sol_file in sol_files:
        sol_file_path = os.path.join(sols_dir, sol_file)
        actions = parse_sol(sol_file_path)
        version_name = sol_file.split(".")[0]
        # Determine the corresponding PDDL file.
        # Eg: logistics/0-0.2/p000146/p000146_0.pddl
        pddl_file_path = None
        for folder in ["0-0.2", "0.2-0.4", "0.4-0.6", "0.6-0.8", "0.8-1.0"]:
            candidate_path = os.path.join(dataset_pddl_base, folder, problem, f"{version_name}.pddl")
            if os.path.exists(candidate_path):
                pddl_file_path = candidate_path
                break
        if pddl_file_path is None:
            print(f"PDDL file not found for problem {version_name} in any subfolder.")
            missing_count+=1
            continue

        initial_state, goals = parse_pddl(pddl_file_path)

        # Create a Plan object and add to the list.
        temp.append(Plan(problem_id=problem, plan_name=version_name, initial_state=initial_state, actions=actions, goals=goals))
        
    if len(temp) == 6:
        plans.extend(temp)
    else:
        print(f"{problem} has missing versions")

# Print out all collected plans
for plan in plans:
    print(plan)
    print("=" * 40)
    
print(len(plans))
print(missing_count)

['p067035_0.SOL', 'p067035_3.SOL', 'p067035_4.SOL', 'p067035_2.SOL', 'p067035_1.SOL', 'p067035_og.SOL']
['p054255_1.SOL', 'p054255_0.SOL', 'p054255_3.SOL', 'p054255_4.SOL', 'p054255_og.SOL', 'p054255_2.SOL']
['p000950_1.SOL', 'p000950_0.SOL', 'p000950_4.SOL', 'p000950_og.SOL', 'p000950_2.SOL', 'p000950_3.SOL']
PDDL file not found for problem p000950_1 in any subfolder.
PDDL file not found for problem p000950_0 in any subfolder.
PDDL file not found for problem p000950_4 in any subfolder.
PDDL file not found for problem p000950_og in any subfolder.
PDDL file not found for problem p000950_2 in any subfolder.
PDDL file not found for problem p000950_3 in any subfolder.
p000950 has missing versions
['p033265_2.SOL', 'p033265_4.SOL', 'p033265_og.SOL', 'p033265_0.SOL', 'p033265_3.SOL', 'p033265_1.SOL']
PDDL file not found for problem p033265_2 in any subfolder.
PDDL file not found for problem p033265_4 in any subfolder.
PDDL file not found for problem p033265_og in any subfolder.
PDDL file not

In [37]:
print(len(plans))
plans_copy = list(plans)
plans_to_drop = []
for p in plans:
    if p.actions == []:
        for plan in plans:
            if p.problem_id == plan.problem_id:
                try:
                    plans_copy.remove(plan)
                except ValueError as e:
                    pass

plans = list(plans_copy)
print(len(plans))

for p in plans:
    if p.actions == []:
        print("Missed")

3210
3180


In [38]:
#create action and goal dictionaries
import pickle
unique_actions = set()
for plan in plans:
    unique_actions.update(plan.actions)
        
unique_actions = list(unique_actions)

print(len(unique_actions))

action_dict = {}

for i, action in enumerate(unique_actions):
    action_dict[action] = i
    
print(action_dict)


unique_goals = set()
for plan in plans:
    unique_goals.update(plan.goals)

# Sort to have a consistent ordering
unique_goals = sorted(unique_goals)
num_goals = len(unique_goals)
goal_dict = {}

# Build the one-hot encoding for each goal
for idx, goal in enumerate(unique_goals):
    onehot = [0] * num_goals
    onehot[idx] = 1
    goal_dict[goal] = onehot

# Print the goal dictionary
print(goal_dict)

with(open("../datasets/gr_logistics/pickles/action_dict.pkl", "wb")) as ad:
    pickle.dump(action_dict, ad)
    
with(open("../datasets/gr_logistics/pickles/goal_dict.pkl", "wb")) as gd:
    pickle.dump(goal_dict, gd)

9078
{'unload-airplane obj55 apn7 apt6': 0, 'drive-truck tru3 pos12 pos11 cit5': 1, 'load-truck obj77 tru1 apt7': 2, 'unload-airplane obj12 apn7 apt5': 3, 'fly-airplane apn7 apt5 apt1': 4, 'load-truck obj21 tru2 apt4': 5, 'drive-truck tru4 apt5 pos12 cit1': 6, 'unload-truck obj77 tru5 pos66': 7, 'drive-truck tru4 apt6 pos55 cit2': 8, 'drive-truck tru5 pos77 apt4 cit6': 9, 'load-truck obj12 tru4 pos12': 10, 'unload-airplane obj77 apn7 apt4': 11, 'drive-truck tru4 apt8 pos55 cit1': 12, 'drive-truck tru2 pos44 pos11 cit3': 13, 'unload-airplane obj88 apn3 apt1': 14, 'drive-truck tru3 apt2 pos11 cit5': 15, 'unload-truck obj55 tru4 apt5': 16, 'drive-truck tru1 pos66 apt4 cit2': 17, 'drive-truck tru1 apt4 pos13 cit5': 18, 'unload-truck obj21 tru3 pos55': 19, 'drive-truck tru3 apt7 pos77 cit1': 20, 'fly-airplane apn8 apt5 apt4': 21, 'unload-airplane obj66 apn2 apt1': 22, 'unload-truck obj77 tru3 pos77': 23, 'drive-truck tru1 pos77 pos13 cit1': 24, 'drive-truck tru5 pos22 pos12 cit5': 25, 'unlo

In [39]:
import random

#now need to pickle train, val, test plans
# Shuffle the plans list to randomize the order
random.shuffle(plans)
n = len(plans)
train_end = int(n * 0.8)
val_end = train_end + int(n * 0.10)

train_plans = plans[:train_end]
val_plans = plans[train_end:val_end]
test_plans = plans[val_end:]

print(len(train_plans))
print(len(val_plans))
print(len(test_plans))

with open("../datasets/gr_logistics/pickles/train_plans.pkl", "wb") as f:
    pickle.dump(train_plans, f)

with open("../datasets/gr_logistics/pickles/val_plans.pkl", "wb") as f:
    pickle.dump(val_plans, f)

with open("../datasets/gr_logistics/pickles/test_plans.pkl", "wb") as f:
    pickle.dump(test_plans, f)


2544
318
318
